In [4]:
import yfinance as yf
import pandas as pd

def fetch_stock_data(ticker, period="1y", interval="1h"):
    stock = yf.Ticker(ticker)
    df = stock.history(period=period, interval=interval)
    df.reset_index(inplace=True)
    return df

def fetch_options_data(ticker, expiration_index=0):
    stock = yf.Ticker(ticker)
    expirations = stock.options
    if not expirations:
        raise Exception("No options data available")
    opt_chain = stock.option_chain(expirations[expiration_index])
    calls = opt_chain.calls
    puts = opt_chain.puts
    calls['Type'] = 'Call'
    puts['Type'] = 'Put'
    calls['Ticker'] = ticker
    puts['Ticker'] = ticker
    return pd.concat([calls, puts])
    
if __name__ == "__main__":
    ticker = "NVDA"
    df_stock = fetch_stock_data(ticker)
    print(df_stock.head())

     

                   Datetime       Open       High        Low      Close  \
0 2024-03-08 09:30:00-05:00  94.800003  97.209999  94.620003  97.209999   
1 2024-03-08 10:30:00-05:00  97.223999  97.400002  90.159996  90.497803   
2 2024-03-08 11:30:00-05:00  90.503853  91.460999  88.057999  89.579056   
3 2024-03-08 12:30:00-05:00  89.577499  90.084000  87.077003  87.606003   
4 2024-03-08 13:30:00-05:00  87.586319  88.488998  86.505997  88.140999   

     Volume  Dividends  Stock Splits  
0  21881251        0.0           0.0  
1  25042101        0.0           0.0  
2  21364970        0.0           0.0  
3  12407589        0.0           0.0  
4  11312076        0.0           0.0  


In [5]:
df_stock

,Datetime,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2024-03-08 09:30:00-05:00,94.800003,97.209999,94.620003,97.209999,21881251,0.0,0.0
1,2024-03-08 10:30:00-05:00,97.223999,97.400002,90.159996,90.497803,25042101,0.0,0.0
2,2024-03-08 11:30:00-05:00,90.503853,91.460999,88.057999,89.579056,21364970,0.0,0.0
3,2024-03-08 12:30:00-05:00,89.577499,90.084000,87.077003,87.606003,12407589,0.0,0.0
4,2024-03-08 13:30:00-05:00,87.586319,88.488998,86.505997,88.140999,11312076,0.0,0.0
...,...,...,...,...,...,...,...,...
1733,2025-03-07 11:30:00-05:00,109.705002,110.000000,107.559998,109.004997,51020128,0.0,0.0
1734,2025-03-07 12:30:00-05:00,109.000000,111.290001,108.699997,110.508904,45996375,0.0,0.0
1735,2025-03-07 13:30:00-05:00,110.529999,112.514999,110.330002,112.250000,34597993,0.0,0.0
1736,2025-03-07 14:30:00-05:00,112.260002,112.919998,111.294998,112.605003,32459442,0.0,0.0


In [7]:

import numpy as np
import pandas as pd
from datetime import datetime
from scipy.stats import norm


# 1. Calculate Log Returns & Historical Volatility (HV)
def calculate_log_returns(df):
    """
    Calculate log returns based on closing prices.
    """
    df['log_return'] = np.log(df['Close'] / df['Close'].shift(1))
    df.dropna(inplace=True)
    return df

def calculate_hv(df, window=30):
    """
    Calculate historical volatility (annualized) over a rolling window.
    """
    # Annualize by multiplying by sqrt(252) assuming 252 trading days
    df['HV'] = df['log_return'].rolling(window=window).std() * np.sqrt(252)
    return df

# 2. Option Greeks using Black-Scholes formula
def black_scholes_greeks(S, K, T, r, sigma, option_type="call"):
    """
    Calculate option Greeks: Delta, Gamma, Theta, Vega, and Rho.
    
    Parameters:
    - S: Underlying stock price
    - K: Strike price
    - T: Time to expiry (in years)
    - r: Risk-free rate (annualized, e.g., 0.02 for 2%)
    - sigma: Implied volatility (as a decimal, e.g., 0.3 for 30%)
    - option_type: 'call' or 'put'
    
    Returns:
    A tuple: (delta, gamma, theta, vega, rho)
    """
    # Avoid division by zero if T is 0
    if T <= 0:
        return np.nan, np.nan, np.nan, np.nan, np.nan

    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    if option_type.lower() == "call":
        delta = norm.cdf(d1)
        theta = (-S * norm.pdf(d1) * sigma / (2 * np.sqrt(T)) -
                 r * K * np.exp(-r * T) * norm.cdf(d2))
        rho = K * T * np.exp(-r * T) * norm.cdf(d2)
    else:
        delta = norm.cdf(d1) - 1
        theta = (-S * norm.pdf(d1) * sigma / (2 * np.sqrt(T)) +
                 r * K * np.exp(-r * T) * norm.cdf(-d2))
        rho = -K * T * np.exp(-r * T) * norm.cdf(-d2)
        
    gamma = norm.pdf(d1) / (S * sigma * np.sqrt(T))
    vega = S * np.sqrt(T) * norm.pdf(d1)
    
    return delta, gamma, theta, vega, rho

# 3. Time to Expiry (T)
def calculate_time_to_expiry(expiry_date, current_date=None):
    """
    Calculate time to expiry in years.
    
    Parameters:
    - expiry_date: Expiration date as a string in 'YYYY-MM-DD' format or a datetime object.
    - current_date: Current date as a datetime object (defaults to now).
    
    Returns:
    - T: Time to expiry in years.
    """
    if current_date is None:
        current_date = datetime.now()
    if isinstance(expiry_date, str):
        expiry_date = datetime.strptime(expiry_date, "%Y-%m-%d")
    days_to_expiry = (expiry_date - current_date).days
    T = days_to_expiry / 365.0
    # If option is expired, T should be zero
    return max(T, 0)

# 4. Moneyness
def calculate_moneyness(S, K):
    """
    Calculate moneyness as the ratio of the underlying stock price to the strike price.
    """
    return S / K


In [11]:
df_options.apply(calculate_log_returns)

KeyError: 'Close'

In [1]:
df_stock

NameError: name 'df_stock' is not defined